In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#define path
wesley_ds = "/files"
new_ds = "/Msc2"
kaggle_ds = "/archive/Data/genres_original"
main_path = ""

In [2]:
#choose the dataset
chooser = new_ds

In [3]:
DATASET_PATH = main_path+chooser
# DATASET_PATH = "../../../Music"
JSON_PATH_NEW = main_path + '' + 'complete_3_first_specTonnetz_Msc2.json'
JSON_PATH = main_path + '' + 'complete_3_first_specTonnetz.json'
SAMPLE_RATE = sr =  22050
DURATION = 30 #measured in seconds 
SAMPLES_PER_TRACK = SAMPLE_RATE*DURATION

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(main_path+chooser):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        print(filename)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [5]:
np.__version__

'1.19.5'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
from IPython.display import Audio
from tqdm import tqdm 
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LSTM, Bidirectional, GRU, BatchNormalization, LeakyReLU
from tensorflow.keras.utils import to_categorical
import numpy as np
# import torch
# import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
import os
import math
import json
import random
import mutagen

In [ ]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=4084, hop_length=1024, num_segments=10):
    #dictionary to store data
    data = {
        'mapping' : [],
        'mfcc' : [],
        'spectral': [],
        'labels' : [],
        'tonnetz': [],
        'key': []
    }
    
    count = 0 # To keep track of our progress
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments) 
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    #Loop through all the genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        #ensure that we're not at the root level
        if dirpath not in dataset_path:

            #save the semantic label
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            data['mapping'].append(semantic_label)
            print('\nProcessing {}'.format(semantic_label))
            
            #process files for a specific genre 
            for f in filenames:
#                 if f.endswith('.wav') and f != 'jazz.00054.wav': # Since file jazz.00054.wav is an empty file
                if f.endswith('.mp3') or f.endswith('.MP3') or True: # Since file jazz.00054.wav is an empty file
                    file_path = os.path.join(dirpath,f)
                    #loading the audio file 
                    # we are using the soundfile library since it is faster than librosa
                    signal, sr = librosa.load(file_path) # len(signal) = 661794  # sr is 22050 by default
                    # try:
                    #     temp1 = str(mutagen.File(file_path)['TXXX:EnergyLevel'])
                    #     temp2 = str(mutagen.File(file_path)['TKEY'])
                    # except:
                    #     continue
                    midptr = int(signal.shape[0]/2)
                    bptr = midptr - (SAMPLE_RATE*int(DURATION/2)) #set as OFFSET
#                     hptr = midptr + (SAMPLE_RATE*duration)
                    #process segments extracting mfcc and storing data
                    for s in range(num_segments): 
                        # Since num_segments is defined as 5. Every 30 sec file is divided into 5 segments of length 6sec 
                        # Start sample would keep track of the index of the first element of each 6 second batch
                        # finish sample would keep track of the index of the last element of each 6 second batch
                        # And then with the help of python's slice functionality we will extract that 6 second batch from every 30 sec signal
                        start_sample = 0 + num_samples_per_segment * s   
                        finish_sample = num_samples_per_segment + start_sample
                        
                        # Next, we will pass each segment in order to extract MFCC. The parameter n_mfcc defines the number of mfcc 
                        # we need to extract, Usually n_mfcc is set b/w 13 to 40. The other parameters n_fft and hop length are 
                        # indivisual topics of discussion. Will be discussed in later Notbooks. 
                        mfcc = librosa.feature.mfcc(signal[start_sample : finish_sample],
                                                   sr = sr,
                                                   n_fft = n_fft,
                                                   n_mfcc = n_mfcc,
                                                   hop_length = hop_length)

                        mfcc = mfcc.T

                        # specconv = librosa.feature.spectral_contrast(signal[start_sample : finish_sample],
                        #                            sr = sr,
                        #                            n_fft = n_fft,
                        #                            hop_length = hop_length)
                        
                        # specconv = specconv.T

                        specconv = librosa.feature.melspectrogram(signal[start_sample : finish_sample],
                                                   sr = sr,
                                                   n_fft = n_fft,
                                                   hop_length = hop_length)
                        
                        specconv = specconv.T

                        tonnetz = librosa.feature.tonnetz(signal[start_sample : finish_sample],
                                                   sr = sr,
                                                   hop_length = hop_length)
                        
                        tonnetz = tonnetz.T
                        # store mfcc for segment if it has the expected length
                        if (( len(mfcc) == expected_num_mfcc_vectors_per_segment )):
                            # print(mfcc.shape)
                            data['mfcc'].append(mfcc.tolist())
                            data['labels'].append(i)
                            # data['energy'].append(temp1)
                            data['key'].append(i)
                            data['spectral'].append(specconv.tolist())
                            data['tonnetz'].append(tonnetz.tolist())
                            print('Processing {}, segment:{}'.format(file_path, s))
                            count += 1
                            print(count)
      
    with open(json_path, 'w') as fp:
        json.dump(data, fp, indent=4)

In [ ]:
# Let's run the above function 
save_mfcc(DATASET_PATH, JSON_PATH_NEW, num_segments=10)

In [42]:
# loading the saved Json file
def load_data(path):
    with open(path, 'r') as fp:
        data = json.load(fp)
    #Convert lists into numpy arrays
    # inputs = np.dstack(( data['spectral'], data['tonnetz']))
    # inputs = data['spectral']
    # inputs = [librosa.power_to_db(np.array(isi).T, ref=np.max).T for isi in data['spectral']]
    inputs = np.dstack(([librosa.power_to_db(np.array(isi).T, ref=np.max).T for isi in data['spectral']], data['tonnetz']))
    targets = data['key'] 
    return np.array(inputs), np.array(targets)

In [8]:
with open(JSON_PATH, 'r') as fp:
    data = json.load(fp)
    print(data['mapping'])

['BIG ROOM', 'DEEP HOUSE', 'BASS HOUSE', 'FUTURE HOUSE', 'ELECTRO HOUSE']


In [9]:
inputs, targets = load_data(JSON_PATH)

In [10]:
#Change mapping to follow same pattern on config file
MAPPER = ['BIG ROOM', 'DEEP HOUSE', 'BASS HOUSE', 'FUTURE HOUSE', 'ELECTRO HOUSE']
targets = [MAPPER[isi-1] for isi in targets]
MAPPER = ['DEEP HOUSE', 'FUTURE HOUSE','BASS HOUSE',  'ELECTRO HOUSE', 'BIG ROOM']
targets = [MAPPER.index(isi) for isi in targets]

In [43]:
inputs, targets = load_data(JSON_PATH)
inputs2, targets2 = load_data(JSON_PATH_NEW)
#convert folder to key name
remapper = ['8B', '1B', '3B']
targets3 = np.array([remapper[isi-1] for isi in targets2])
targets2 = targets3
inputs3 = np.vstack((inputs,inputs2))
targets3 = np.hstack((targets, targets2))
inputs = inputs3
targets = targets3

In [44]:
print(inputs.shape)

(7250, 65, 134)


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
# S_db = librosa.power_to_db(inputs[0,:,:].T, ref=np.max)
img = librosa.display.specshow(inputs[-1,:,:].T, x_axis ='time', y_axis='mel')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency spectrogram')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
signal, sr = librosa.load(DATASET_PATH+'/BIG ROOM/01 Bigenesis _ Jadon Fonka - Fluto (Extended.mp3')
specconv = librosa.feature.melspectrogram(signal[0 : 22050*3], sr=22050, n_fft=4084, hop_length=1024)
S_db = librosa.power_to_db(specconv, ref=np.max)
print(S_db.shape)
img = librosa.display.specshow(specconv, x_axis ='time', y_axis='mel')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency spectrogram')

In [ ]:
targets = np.array([isi-1 for isi in targets])

In [45]:
np.unique(targets, return_counts=True)

(array(['10A', '10B', '11A', '11B', '12A', '12B', '1A', '1B', '2A', '2B',
        '3A', '3B', '4A', '4B', '5A', '5B', '6A', '6B', '7A', '7B', '8A',
        '8B', '9A', '9B'], dtype='<U3'),
 array([ 130,   20,  960,   20,  220,   50,  490,  200,  300,   30,  120,
         200, 1710,   30,  250,   50,  870,   30,  220,   20,  310,  210,
         780,   30], dtype=int64))

In [46]:
outp_key = ['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B']
#for key purpose
targets2 = np.array([outp_key.index(isi) for isi in targets])
print(targets2)

for isi in targets:
    print(outp_key[outp_key.index(isi)], isi)

[ 5  5  5 ... 14 14 14]
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
7A 7A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A

4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
8A 8A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
3A 3A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A

6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
2A 2A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
11A 11A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A


4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
9A 9A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
1A 1A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
5A 5A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
6A 6A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
4A 4A
10A 10A
10A 10A
10A 10A
10A 10A
10A 10A
10A 10A
10A 

In [47]:
print(np.unique(targets2, return_counts=True))
print(len(outp_key))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23]), array([ 490,  300,  120, 1710,  250,  870,  220,  310,  780,  130,  960,
        220,  200,   30,  200,   30,   50,   30,   20,  210,   30,   20,
         20,   50], dtype=int64))
24


In [ ]:
# Keras Model

In [48]:
inputs.shape

(7250, 65, 134)

In [66]:
model = Sequential()


# model.add(GRU(800, input_shape=(inputs.shape[1], inputs.shape[2])))

model.add(GRU(600, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2])))
# model.add(GRU(200, return_sequences=True))
model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(GRU(300))

model.add(Dropout(0.5))
# model.add(BatchNormalization())

model.add(Dense(24, 'softmax'))
# model.add(Dense(5, 'sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])



model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_8 (GRU)                  (None, 65, 600)           1324800   
_________________________________________________________________
dropout_8 (Dropout)          (None, 65, 600)           0         
_________________________________________________________________
gru_9 (GRU)                  (None, 300)               811800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                7224      
Total params: 2,143,824
Trainable params: 2,143,824
Non-trainable params: 0
_________________________________________________________________


In [51]:
#Spectral Whitening
from sklearn.preprocessing import StandardScaler
nw_arr = []
for isi in inputs[:,:,0:128]:
  scaler = StandardScaler()
  nw_arr.append(scaler.fit_transform(isi.T).T) # Do Averaging on each frequency band, across time-domain, then transpose to time-axis again
print(np.array(nw_arr).shape)
inputs = np.dstack((np.array(nw_arr), inputs[:,:,128:134])) # do this if want to combine with rest feature
# inputs = np.array(nw_arr) # do this if only want to use melspectogram
print(inputs.shape)

(7250, 65, 128)
(7250, 65, 134)


In [60]:
#Per Sample STDD

nw_arr = []
for isi in inputs[:,:,0:128]:
  avg = np.mean(isi, dtype=np.float64)
  stds = np.std(isi, dtype=np.float64)
  nw_arr.append((isi-avg)) # Do Averaging on each frequency band, across time-domain, then transpose to time-axis again
print(np.array(nw_arr).shape)
inputs = np.dstack((np.array(nw_arr), inputs[:,:,128:134])) # do this if want to combine with rest feature
# inputs = np.array(nw_arr) # do this if only want to use melspectogram
print(inputs.shape)

(7250, 65, 128)
(7250, 65, 134)


In [ ]:
#x10scaler
inputs = np.dstack((inputs[:,:,0:128]*10, inputs[:,:,128:134])) #as only first 128 features are Melspectogram
print(inputs.shape)

In [61]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets2, test_size=0.1)

In [62]:
np.unique(targets_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 442,  270,  109, 1550,  223,  787,  196,  288,  700,  111,  875,
         192,  174,   25,  178,   29,   45,   27,   17,  179,   27,   20,
          19,   42], dtype=int64))

In [63]:
inputs_train, targets_train = augment_ds(inputs_train, targets_train)

(13050, 65, 134) (13050,)


In [64]:
inputs_test.shape

(725, 65, 134)

In [67]:
history = model.fit(inputs_train, targets_train,
          validation_data=(inputs_test, targets_test),
          epochs = 10,
          batch_size=100)

Epoch 1/10
131/131 [==============================] - 95s 721ms/step - loss: 1.9980 - accuracy: 0.4146 - val_loss: 1.3670 - val_accuracy: 0.5959
Epoch 2/10
131/131 [==============================] - 93s 708ms/step - loss: 1.0882 - accuracy: 0.6679 - val_loss: 1.0488 - val_accuracy: 0.6593
Epoch 3/10
131/131 [==============================] - 94s 721ms/step - loss: 0.6319 - accuracy: 0.7994 - val_loss: 0.6349 - val_accuracy: 0.8055
Epoch 4/10
131/131 [==============================] - 92s 704ms/step - loss: 0.3742 - accuracy: 0.8809 - val_loss: 0.5396 - val_accuracy: 0.8441
Epoch 5/10
131/131 [==============================] - 90s 690ms/step - loss: 0.2546 - accuracy: 0.9202 - val_loss: 0.4689 - val_accuracy: 0.8717
Epoch 6/10
131/131 [==============================] - 98s 747ms/step - loss: 0.1348 - accuracy: 0.9569 - val_loss: 0.4755 - val_accuracy: 0.8855
Epoch 7/10
131/131 [==============================] - 96s 735ms/step - loss: 0.1011 - accuracy: 0.9688 - val_loss: 0.4322 - val_ac

In [68]:
model.save(main_path+'model/my_model_key')


INFO:tensorflow:Assets written to: model/my_model_key\assets


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
print(tf.argmax(model(inputs_test), axis=1))

In [ ]:
### Model Analysis
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
cf_matrix = confusion_matrix(targets_test, tf.argmax(model(inputs_test), axis=1))
make_confusion_matrix(cf_matrix, figsize=(24,24), cbar=False)
print(classification_report(targets_test, tf.argmax(model(inputs_test), axis=1)))

# plot_confusion_matrix(confusion_matrix=confusion_matrix(targets2, tf.argmax(model(inputs), axis=1)),display_labels=[isi for isi in range(24)]).plot()


In [ ]:
# Pytorch

In [19]:
def augment_ds(inputs, outputs):
    inputs_cpy = np.array(inputs)
    inputs_ori = np.array(inputs)
    outputs_cpy = np.array(outputs)
    # Adding Noise 
    
    for i in range(inputs_cpy.shape[0]):
        s = np.random.rand(inputs_cpy.shape[1], inputs_cpy.shape[2])
        inputs_cpy[i] = inputs_cpy[i] + s

    #join dataset
    inputs_cpy = np.vstack((inputs_cpy, inputs_ori))
    outputs_cpy = np.hstack((outputs, outputs))
    print(inputs_cpy.shape, outputs_cpy.shape)
    return inputs_cpy, outputs_cpy

In [ ]:
# Model Definition
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, prob_do):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.do = nn.Dropout(p=prob_do)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.do(out)
        out = torch.sigmoid(self.fc(out))
        return out

In [ ]:
def create_models(input_size, hidden_size, num_classes, device, prob_do = 0.5, num_layers = 1):
    return RNN(input_size, hidden_size, num_layers, num_classes, prob_do).to(device)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

def train_models(num_epoch, batch_size, learning_rate, inputs, outputs, splitts, device, folds = 0):
    train_index, test_index = splitts[folds]
    x_train = inputs[train_index]
    y_train = outputs[train_index]
    
    x_train, y_train = augment_ds(x_train, y_train)
    x_test = inputs[test_index]
    y_test = outputs[test_index]
    
    models = create_models(inputs.shape[-1], 600, 5, device, prob_do = 0.4)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(models.parameters(), lr=learning_rate)
    
    x_train = torch.from_numpy(x_train)
    y_train = torch.from_numpy(y_train)
    x_test = torch.from_numpy(x_test)
    y_test = torch.from_numpy(y_test)
#     x_val = torch.from_numpy(x_val)
#     y_val = torch.from_numpy(y_val)
    
    print(x_train.shape)
    train_loss = []
    test_loss = []
    acc_arr = []
    train = TensorDataset(x_train, y_train)
    test = TensorDataset(x_test, y_test)
#     val = TensorDataset(x_val, y_val)
    
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
#     val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)
#     test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)
    for epoch in range(num_epoch):
        # training sequences
        batch_losses = []
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.view([batch_size, -1, inputs.shape[-1]]).to(device).to(torch.float32)
            y_batch = y_batch.to(device).to(torch.int64)
            
            models.train()
            y_hat = models(x_batch)
            loss = criterion(y_hat, y_batch)

            # Backward and optimize
            loss.backward()
            # Updates parameters and zeroes gradients
            optimizer.step()
            optimizer.zero_grad()
            
            batch_losses.append(loss.item())
        train_loss.append(np.mean(batch_losses))
        
        #testing sequences
        with torch.no_grad():
            batch_val_losses = []
            for x_val, y_val in test_loader:
                x_val = x_val.view([batch_size, -1, inputs.shape[-1]]).to(device).to(torch.float32)
                y_val = y_val.to(device).to(torch.int64)
                models.eval()
                yhat = models(x_val)
                val_loss = criterion(yhat, y_val)
                batch_val_losses.append(val_loss.item())
            test_loss.append(np.mean(batch_val_losses))
        
        print(f"[{epoch+1}/{num_epoch}] Training loss: {train_loss[-1]:.4f}\t Validation loss: {test_loss[-1]:.4f}", end="\n")
            
    return models,train_loss, test_loss, acc_arr

In [ ]:
def evaluation(models, test_loader,device, batch_size):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        batch_acc = []
        #labels = labels.to(device)
        for x_test, y_test in test_loader:
            x_test = x_test.view([batch_size, -1, x_test.shape[-1]]).to(device)
            y_test = y_test.to(device)
            x_test = x_test.to(torch.float32)
            y_test = y_test.to(torch.int64)
            models.eval()  
            outputs = models(x_test)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += y_test.size(0)
#             print(y_test)
            n_correct += (predicted == y_test).sum().item()
            acc = 100.0 * n_correct / n_samples
            batch_acc.append(acc)
#         print(n_correct)
        print(f'Accuracy: {np.mean(batch_acc) :.4f} %')

In [ ]:
inputs, targets = load_data(JSON_PATH)

In [ ]:
#device conf
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 5 #deprecated
num_epochs = 30
batch_size = 100
learning_rate = 0.001

hidden_size = 500 #deprecated
num_layers = 1 #deprecated

from sklearn.model_selection import KFold as StratifiedKFold
# manage dataset

# split training and testing
x_train, x_val, y_train, y_val = train_test_split(inputs, targets, test_size=0.1)
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)
val = TensorDataset(x_val, y_val)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

skf = StratifiedKFold(n_splits=10)
output_lst = ["BASS HOUSE", "BIG ROOM", "DEEP HOUSE", "ELECTRO HOUSE", "FUTURE HOUSE"]
outp_key = ['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B']

#for genre purpose
targets = np.array([isi-1 for isi in targets])
print(targets.shape)
row_skf = [content for content in skf.split(x_train, y_train)]



In [ ]:
inputs.shape

In [ ]:
# for index in range(10):
#     model, tl, tsl, acr = train_models(num_epochs, batch_size, learning_rate, x_train, y_train, row_skf, device, folds = index)
#     evaluation(model, val_loader,device, batch_size)
# # evaluation(model, ldr,device, batch_size)

model, tl, tsl, acr = train_models(num_epochs, batch_size, learning_rate, x_train, y_train, row_skf, device, folds = 0)
evaluation(model, val_loader,device, batch_size)

In [ ]:
model.eval()
torch.save(model.state_dict(), '81_3_genre_inf.pth')

In [ ]:
#optional: export as ONNX model
model.eval()
batch_sizer = 1
x = torch.randn(batch_sizer, 65, 13, requires_grad=True).to(device)

torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "Genre__cls__81_3.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
predictions_one_hot = model.predict(inputs_test)
print(targets_test.shape, predictions_one_hot.shape)
# cm = confusion_matrix(targets_test, predictions_one_hot.argmax(axis=1))
# print(cm)
con_mat = tf.math.confusion_matrix(labels=targets_test, predictions=predictions_one_hot.argmax(axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = outp_key, 
                     columns = outp_key)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
signal, sr = librosa.load("../raw_dataset/files/BIG ROOM/01 Agus Zack _ Devbanz - Workout (Extended M.mp3") # len(signal) = 661794  # sr is 22050 by default 

In [ ]:
signal.shape[0]
mfcc = librosa.feature.mfcc(signal[0 : 22050*30],
                                                   sr = 22050,
                                                   n_fft = 4084,
                                                   n_mfcc = 13,
                                                   hop_length = 1024)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# (rate,sig) = wav.read("file.wav")
mfcc_feat = mfcc

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_feat, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower', aspect='auto')
ax.set_title('MFCC')
#Showing mfcc_data
plt.show()
#Showing mfcc_feat
plt.plot(mfcc_feat)
plt.show()

In [ ]:
mfcc.shape

In [ ]:
# Clustering

In [ ]:
!pip install tslearn

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
km = TimeSeriesKMeans(n_clusters=len(outp_key), metric="euclidean")
km.fit(inputs_train)

In [ ]:
# cluster rename

In [ ]:
nonexist_val = [12,14,19] #list ground truth dont have data references

cluster_nwmaps = [-1 for isi in outp_key] # mapping cluster -> real output

for isi in np.unique(targets_train, return_counts=True)[0]:
  # print(inputs_train[targets_train.tolist().index(isi),:,:].shape)

  tmps = km.predict(inputs_train[targets_train.tolist().index(isi),:,:].reshape(1,65,25))
  r = 0
  while ((r<len(tmps)-1) and (cluster_nwmaps[tmps[r]] != -1)):
    r += 1
  if (cluster_nwmaps[tmps[r]] != -1):
    # print("error")
    # raise ValueError('Two cluster point to same output target')
    nonexist_val.append(isi)
  else:
    cluster_nwmaps[tmps[r]] = isi
r = 0
for isi in range(len(cluster_nwmaps)):
  if (cluster_nwmaps[isi]==-1):
    cluster_nwmaps[isi] = nonexist_val[r] # map the rest of cluster that dont have reference as nonexist_val
    r += 1

In [ ]:
# eval cluster

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(targets_test, [cluster_nwmaps[isi] for isi in km.predict(inputs_test)]))